In [1]:
import math
import numpy as np
import wandb

import torch
import torch_geometric
from torch_geometric.data import Data

from gnn_architectures import MyGnn

import gnn_io as gio
import gnn_architectures as garch

import pprint

## 1. Define model and parameters

In [2]:
wandb.login()

# Define parameters 
num_epochs = 1000
project_name = 'test_feature_base_vol_car'
train_ratio = 0.8

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

early_stopping = gio.EarlyStopping(patience=10, verbose=True)
torch.set_printoptions(precision=4, sci_mode=False)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: enatterer (tum-traffic-engineering). Use `wandb login --relogin` to force relogin


## 2. Load data

In [3]:
# Load the list of dictionaries
data_dict_list = torch.load('../data/dataset_1pm_0-1400_more_features_.pt')

# Reconstruct the Data objects
datalist = [Data(x=d['x'], edge_index=d['edge_index'], pos=d['pos'], y=d['y'] - d['x'][:, 1]) for d in data_dict_list]

In [ ]:
datalist[0]

Data(x=[31216, 6], edge_index=[2, 59135], y=[31216, 1], pos=[31216, 2])

In [ ]:
datalist[0].x

tensor([[   480.0000,      7.3571,      4.0000,      8.3333,     16.1813,
              4.0000],
        [   240.0000,      9.2143,      3.0000,      8.3333,     14.8602,
              5.0000],
        [   480.0000,      1.5000,      3.0000,      8.3333,     14.8602,
              5.0000],
        ...,
        [  7999.2002,      0.0000,     -1.0000,     12.0000,    699.8928,
             14.0000],
        [  7999.2002,      0.0000,     -1.0000,     20.0000,     20.0000,
             15.0000],
        [  7999.2002,      0.0000,     -1.0000,      6.0000,    611.8889,
             16.0000]])

In [ ]:
asdfs

In [ ]:
datalist[0].y

In [ ]:
asdfasa

In [ ]:

dataset_only_relevant_dimensions = gio.cut_dimensions(dataset=datalist, indices_of_dimensions_to_keep=[0, 1, 2])
dataset_normalized = gio.normalize_dataset(dataset_only_relevant_dimensions)

In [ ]:
baseline_error = gio.compute_baseline_error(dataset_normalized)
print(f'Baseline error: {baseline_error}')

Baseline error: 0.005564325489103794


## 4. Train the model

We first find a good model for one batch. 

In [ ]:
def train(model, config=None, loss_fct=None, optimizer=None, train_dl=None, valid_dl=None, device=None, early_stopping=None):
    for epoch in range(config.epochs):
        model.train()
        for idx, data in enumerate(train_dl):
            input_node_features, targets = data.x.to(device), data.y.to(device)
            optimizer.zero_grad()

            # Forward pass
            predicted = model(data)
            train_loss = loss_fct(predicted, targets)
            
            # Backward pass
            train_loss.backward()
            optimizer.step()
            
            wandb.log({"train_loss": train_loss.item(), "epoch": epoch, "step": idx})
            # print(f"epoch: {epoch}, step: {idx}, loss: {train_loss.item()}")
        
        val_loss = garch.validate_model_pos_features(model, valid_dl, loss_fct, device)
        print(f"epoch: {epoch}, validation loss: {val_loss}")
        wandb.log({"loss": val_loss, "epoch": epoch})
            
        early_stopping(val_loss)
        if early_stopping.early_stop:
            print("Early stopping triggered. Stopping training.")
            break
    
    print("Best validation loss: ", val_loss)
    wandb.summary["val_loss"] = val_loss
    wandb.finish()
    return val_loss, epoch

In [ ]:
# Currently best architecture ! 

batch_size = 16
output_layer_parameter = 'gat'
hidden_size_parameter = 16
gat_layer_parameter = 0
gcn_layer_parameter = 0
lr = 0.001

wandb.login()

train_dl = gio.create_dataloader(dataset=dataset_normalized, is_train=True, batch_size=batch_size, train_ratio=train_ratio)
valid_dl = gio.create_dataloader(dataset=dataset_normalized, is_train=False, batch_size=batch_size, train_ratio=train_ratio)

wandb.init(
    project=project_name,
    config={
        "epochs": num_epochs,
        "batch_size": batch_size,
        "lr": lr,
        "early_stopping_patience": 10,
        "hidden_layer_size": hidden_size_parameter,
        "gat_layers": gat_layer_parameter,
        "gcn_layers": gcn_layer_parameter,
        "output_layer": output_layer_parameter,
        # "dropout": 0.15,
    }
)
config = wandb.config

print("output_layer: ", output_layer_parameter)
print("hidden_size: ", hidden_size_parameter)
print("gat_layers: ", gat_layer_parameter)
print("gcn_layers: ", gcn_layer_parameter)

gnn_instance = MyGnn(in_channels=3, out_channels=1, hidden_size=hidden_size_parameter, gat_layers=gat_layer_parameter, gcn_layers=gcn_layer_parameter, output_layer=output_layer_parameter)
model = gnn_instance.to(device)

best_val_loss, best_epoch = train(model, config=config, 
                                loss_fct=torch.nn.MSELoss(), 
                                optimizer=torch.optim.Adam(model.parameters(), lr=lr),
                                train_dl=train_dl, valid_dl=valid_dl,
                                device=device, early_stopping=early_stopping)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Total dataset length: 1382
Training subset length: 1104
Total dataset length: 1382
Validation subset length: 272


output_layer:  gat
hidden_size:  16
gat_layers:  0
gcn_layers:  0
Model initialized
MyGnn(
  (pointLayer): PointNetConv(local_nn=Sequential(
    (0): Linear(in_features=3, out_features=16, bias=True)
    (1): ReLU()
    (2): Linear(in_features=16, out_features=16, bias=True)
  ), global_nn=Sequential(
    (0): Linear(in_features=16, out_features=8, bias=True)
    (1): ReLU()
    (2): Linear(in_features=8, out_features=32, bias=True)
    (3): ReLU()
    (4): Linear(in_features=32, out_features=16, bias=True)
  ))
  (output_layer): GATConv(16, 1, heads=1)
)
epoch: 0, validation loss: 0.005555874805020935
epoch: 1, validation loss: 0.005551782765370958
epoch: 2, validation loss: 0.005550890366601593
epoch: 3, validation loss: 0.005549537017941475
epoch: 4, validation loss: 0.00554748865611413
epoch: 5, validation loss: 0.005544354837826069
epoch: 6, validation loss: 0.005535093683968572
epoch: 7, validation loss: 0.005524919724420589
epoch: 8, validation loss: 0.0055134037564344264
epoch:

KeyboardInterrupt: 

In [ ]:
torch.save(model, '../data/trained_models/model_with_features_base_flow_and_highway_classification_target_is_difference.pth')